# Admissions - Mechanical Ventilation

## Step-by-step retrieval and filtering of data

### 1. Imports and database construction

In [4]:
import pandas

from database import SQLConnection
from tqdm import tqdm

from typing import Dict

pandas.options.mode.chained_assignment = None  # default='warn'

sql_server = SQLConnection('../amsterdamumcdb.db')    # open local server

Connection to SQLite DB successful


### 2. Retrieve only ICU admissions

In [6]:
admissions_query = """SELECT * FROM admissions"""
admissions = sql_server.execute_read_query(admissions_query)
admissions_icu = admissions[admissions['location'] == 'IC']
# Remove admissions whose dischargedat = admittedat - Mistakes in the records?
admissions_icu = admissions_icu[admissions_icu['dischargedat'] != 0]
admissions_icu

,patientid,admissionid,admissioncount,location,urgency,origin,admittedat,admissionyeargroup,dischargedat,lengthofstay,destination,gender,agegroup,dateofdeath,weightgroup,weightsource,heightgroup,heightsource,specialty
0,0,0,1,IC,0,None,0,2003-2009,148800000,42,16,Vrouw,80+,NaN,60-69,Anamnestisch,160-169,Anamnestisch,Cardiochirurgie
1,1,1,1,IC,0,None,0,2010-2016,96120000,26,15,Man,60-69,NaN,70-79,Anamnestisch,170-179,Anamnestisch,Cardiochirurgie
2,2,2,1,IC,1,Eerste Hulp afdeling zelfde ziekenhuis,0,2010-2016,84240000,23,15,Man,60-69,NaN,90-99,Anamnestisch,180-189,Anamnestisch,Cardiochirurgie
3,3,3,1,IC,0,None,0,2003-2009,84900000,23,14,Man,50-59,NaN,90-99,None,180-189,Gemeten,Cardiochirurgie
5,5,5,1,IC,1,Eerste Hulp afdeling zelfde ziekenhuis,0,2010-2016,246420000,69,31,Man,50-59,NaN,60-69,Geschat,160-169,Gemeten,Longziekte
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23096,20317,23543,1,IC,0,None,0,2003-2009,76680000,22,15,None,70-79,2.888758e+11,90-99,None,170-179,None,Cardiochirurgie
23099,20320,23546,1,IC,1,Eerste Hulp afdeling zelfde ziekenhuis,0,2010-2016,351900000,98,4,Man,60-69,7.254600e+08,80-89,Geschat,190+,Geschat,Cardiologie
23100,20321,23547,1,IC,1,None,0,2003-2009,267780000,75,Overleden,Vrouw,70-79,2.029800e+08,80-89,Gemeten,160-169,Gemeten,Cardiologie
23103,20324,23550,1,IC,1,Eerste Hulp afdeling zelfde ziekenhuis,0,2010-2016,69720000,20,Overleden,Man,80+,-3.564000e+07,59-,Geschat,170-179,Geschat,Inwendig


### 3. Retrieve admissions with mechanical ventilation involved
Note: Followed from Tariq's SQL queries

Apparently `Beademen` means support, ventilation. We consider only invasive
mechanical ventilation - exclude non-invasief.

Note 2: Why is there `colo-stoma` in processitems' itemid?

In [7]:
is_ventilation_query = """
    SELECT * FROM processitems
    WHERE itemid IN (
        9328, --Beademen
        10731 --Beademen non-invasief
    )
    """
ventilation = sql_server.execute_read_query(is_ventilation_query)
ventilation = ventilation[ventilation['item'] == 'Beademen']
ventilation

,admissionid,itemid,item,start,stop,duration
0,0,9328,Beademen,20520000,69120000,810
1,1,9328,Beademen,2100000,26400000,405
2,2,9328,Beademen,540000,10080000,159
3,3,9328,Beademen,6840000,32040000,420
4,4,9328,Beademen,6900000,35460000,476
...,...,...,...,...,...,...
18830,23543,9328,Beademen,780000,76680000,1265
18831,23546,9328,Beademen,3660000,248160000,4075
18832,23547,9328,Beademen,8580000,267780000,4320
18833,23548,9328,Beademen,4680000,45180000,675


### 4. Combine ICU admissions and ventilation times

In [9]:
from mechanical_ventilation import no_mechvent_admissiontimes
icu_adms_not_mechvent = no_mechvent_admissiontimes(
    icu=admissions_icu,
    vent=ventilation
)
icu_adms_not_mechvent

100%|██████████| 14084/14084 [00:16<00:00, 834.47it/s]


,patientid,admissionid,admissioncount,location,urgency,origin,admittedat,admissionyeargroup,dischargedat,lengthofstay,...,gender,agegroup,dateofdeath,weightgroup,weightsource,heightgroup,heightsource,specialty,init_unventilated,end_unventilated
0,0,0,1,IC,0,None,0,2003-2009,148800000,42,...,Vrouw,80+,NaN,60-69,Anamnestisch,160-169,Anamnestisch,Cardiochirurgie,0,20520000
1,0,0,1,IC,0,None,0,2003-2009,148800000,42,...,Vrouw,80+,NaN,60-69,Anamnestisch,160-169,Anamnestisch,Cardiochirurgie,69120000,148800000
2,1,1,1,IC,0,None,0,2010-2016,96120000,26,...,Man,60-69,NaN,70-79,Anamnestisch,170-179,Anamnestisch,Cardiochirurgie,0,2100000
3,1,1,1,IC,0,None,0,2010-2016,96120000,26,...,Man,60-69,NaN,70-79,Anamnestisch,170-179,Anamnestisch,Cardiochirurgie,26400000,96120000
4,2,2,1,IC,1,Eerste Hulp afdeling zelfde ziekenhuis,0,2010-2016,84240000,23,...,Man,60-69,NaN,90-99,Anamnestisch,180-189,Anamnestisch,Cardiochirurgie,0,540000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31191,20320,23546,1,IC,1,Eerste Hulp afdeling zelfde ziekenhuis,0,2010-2016,351900000,98,...,Man,60-69,725460000.0,80-89,Geschat,190+,Geschat,Cardiologie,248160000,351900000
31192,20321,23547,1,IC,1,None,0,2003-2009,267780000,75,...,Vrouw,70-79,202980000.0,80-89,Gemeten,160-169,Gemeten,Cardiologie,0,8580000
31193,20321,23547,1,IC,1,None,0,2003-2009,267780000,75,...,Vrouw,70-79,202980000.0,80-89,Gemeten,160-169,Gemeten,Cardiologie,267780000,267780000
31194,20325,23551,1,IC,1,None,0,2003-2009,165660000,46,...,Vrouw,40-49,115500000.0,60-69,Geschat,160-169,Geschat,Neurochirurgie,0,3900000


### 5. Close local server

In [10]:
sql_server.close()

Connection to SQLite DB closed


### Visual inspection of some splits

In [19]:
adcols = ['admissionid', 'admittedat', 'dischargedat']
rescols = ['admissionid', 'init_unventilated', 'end_unventilated']

#### a) Case mechanical ventilation is located within admission

In [20]:
admin = 0
print('ICU Admission Data:')
print(admissions_icu[admissions_icu['admissionid'] == admin][adcols])
print('\n> Ventilation times for this admissionid:')
print(ventilation[ventilation['admissionid'] == admin])
print('\n> Computed final valid times:')
print(icu_adms_not_mechvent[icu_adms_not_mechvent['admissionid'] ==
                            admin][rescols])

ICU Admission Data:
   admissionid  admittedat  dischargedat
0            0           0     148800000

> Ventilation times for this admissionid:
   admissionid  itemid      item     start      stop  duration
0            0    9328  Beademen  20520000  69120000       810

> Computed final valid times:
   admissionid  init_unventilated  end_unventilated
0            0                  0          20520000
1            0           69120000         148800000


#### b) Mech. ventilation left even after discharge

In [21]:
admin = 115
print('ICU Admission Data:')
print(admissions_icu[admissions_icu['admissionid'] == admin][adcols])
print('\n> Ventilation times for this admissionid:')
print(ventilation[ventilation['admissionid'] == admin])
print('\n> Computed final valid times:')
print(icu_adms_not_mechvent[icu_adms_not_mechvent['admissionid'] ==
                            admin][rescols])

ICU Admission Data:
     admissionid  admittedat  dischargedat
115          115           0      85500000

> Ventilation times for this admissionid:
    admissionid  itemid      item    start       stop  duration
84          115    9328  Beademen  6300000  104460000      1636

> Computed final valid times:
     admissionid  init_unventilated  end_unventilated
146          115                  0           6300000


#### c) Mech. ventilation started before admission, removed before discharge


In [22]:
admin = 3580
print('ICU Admission Data:')
print(admissions_icu[admissions_icu['admissionid'] == admin][adcols])
print('\n> Ventilation times for this admissionid:')
print(ventilation[ventilation['admissionid'] == admin])
print('\n> Computed final valid times:')
print(icu_adms_not_mechvent[icu_adms_not_mechvent['admissionid'] ==
                            admin][rescols])

ICU Admission Data:
      admissionid  admittedat  dischargedat
3498         3580           0      79140000

> Ventilation times for this admissionid:
      admissionid  itemid      item   start      stop  duration
2810         3580    9328  Beademen -600000  28200000       480

> Computed final valid times:
      admissionid  init_unventilated  end_unventilated
4643         3580           28200000          79140000


#### d) Potential errors? Mech. Vent. connected and removed before admission

In [23]:
admin = 4630
print('ICU Admission Data:')
print(admissions_icu[admissions_icu['admissionid'] == admin][adcols])
print('\nVentilation times for this admissionid:')
print(ventilation[ventilation['admissionid'] == admin])
print('\nComputed final valid times:')
print(icu_adms_not_mechvent[icu_adms_not_mechvent['admissionid'] ==
                            admin][rescols])

ICU Admission Data:
      admissionid  admittedat  dischargedat
4533         4630  1231560000    1256340000

Ventilation times for this admissionid:
      admissionid  itemid      item      start        stop  duration
3683         4630    9328  Beademen  521700000  1222980000     11688

Computed final valid times:
      admissionid  init_unventilated  end_unventilated
6082         4630         1231560000        1256340000
